In [ ]:
import random
import matplotlib.pyplot as plt

class Ship:
    def __init__(self, length):
        self.length = length
        self.positions = set()

class Board:
    def __init__(self, size):
        self.size = size
        self.ships = []
        self.grid = [['O' for _ in range(size)] for _ in range(size)]

    def place_ship(self, ship):
        while True:
            orientation = random.choice(['horizontal', 'vertical'])
            row = random.randint(0, self.size - 1)
            col = random.randint(0, self.size - 1)

            if self.is_valid_placement(ship, row, col, orientation):
                if orientation == 'horizontal':
                    ship.positions = {(row, col + i) for i in range(ship.length)}
                else:
                    ship.positions = {(row + i, col) for i in range(ship.length)}

                self.ships.append(ship)
                for position in ship.positions:
                    self.grid[position[0]][position[1]] = 'S'
                break

    def is_valid_placement(self, ship, row, col, orientation):
        if orientation == 'horizontal':
            positions = {(row, col + i) for i in range(ship.length)}
        else:
            positions = {(row + i, col) for i in range(ship.length)}

        for position in positions:
            if not (0 <= position[0] < self.size and 0 <= position[1] < self.size):
                return False
            if self.grid[position[0]][position[1]] != 'O':
                return False

        return True

    def print_board(self, show_ships=False):
        for row in range(self.size):
            for col in range(self.size):
                if show_ships:
                    has_ship = any((row, col) in ship.positions for ship in self.ships)
                    print('S' if has_ship else self.grid[row][col], end=' ')
                else:
                    print(self.grid[row][col], end=' ')
            print()

    def are_all_ships_sunk(self):
        return all(all(self.grid[position[0]][position[1]] == 'X' for position in ship.positions) for ship in self.ships)

class Player:
    def __init__(self, name):
        self.name = name

    def make_guess(self, board_size):
        raise NotImplementedError("Subclasses must implement the make_guess method.")

class HumanPlayer(Player):
    def make_guess(self, board_size):
        while True:
            try:
                guess_row = int(input(f"{self.name}, guess Row (0-{board_size - 1}): "))
                guess_col = int(input(f"{self.name}, guess Col (0-{board_size - 1}): "))

                if 0 <= guess_row < board_size and 0 <= guess_col < board_size:
                    return (guess_row, guess_col)
                else:
                    print("Please enter valid row and column values.")

            except ValueError:
                print("Please enter valid numeric values.")

class ComputerPlayer(Player):
    def make_guess(self, board_size):
        guess_row = random.randint(0, board_size - 1)
        guess_col = random.randint(0, board_size - 1)
        return (guess_row, guess_col)

class BattleshipGame:
    def __init__(self, board_size):
        self.board_size = board_size
        self.players = []

    def add_player(self, player):
        self.players.append(player)

    def play(self):
        for player in self.players:
            player_board = Board(self.board_size)
            player_board.place_ship(Ship(3))

            if isinstance(player, HumanPlayer):
                print(f"{player.name}'s Board:")
                player_board.print_board(show_ships=True)

            player.opponent_board = Board(self.board_size)
            player.opponent_board.place_ship(Ship(3))

        while True:
            for player in self.players:
                print(f"\n{player.name}'s Turn:")
                guess = player.make_guess(self.board_size)

                if guess in player.opponent_board.ships[0].positions:
                    print("Hit!")
                    player.opponent_board.grid[guess[0]][guess[1]] = 'X'
                    player.opponent_board.print_board()
                    if player.opponent_board.are_all_ships_sunk():
                        print(f"\n{player.name} wins!")
                        loser = next(p for p in self.players if p != player)
                        print(f"{loser.name} loses!")
                        return
                else:
                    print("Miss!")
                    player.opponent_board.grid[guess[0]][guess[1]] = 'M'
                    player.opponent_board.print_board()


def run_computer_vs_computer_stats(board_size, num_games):
    results = {'A1': 0, 'B2': 0, 'C3': 0, 'D4': 0, 'E5': 0}

    for _ in range(num_games):
        computer_player1 = ComputerPlayer("Computer 1")
        computer_player2 = ComputerPlayer("Computer 2")

        game = BattleshipGame(board_size=board_size)
        game.add_player(computer_player1)
        game.add_player(computer_player2)

        first_moves = ['A1', 'B2', 'C3', 'D4', 'E5']
        move_player1 = random.choice(first_moves)
        first_moves.remove(move_player1)
        move_player2 = random.choice(first_moves)

        computer_player1.opponent_board = Board(board_size)
        computer_player2.opponent_board = Board(board_size)

        computer_player1.opponent_board.grid[ord(move_player1[0]) - 65][int(move_player1[1]) - 1] = 'M'
        computer_player2.opponent_board.grid[ord(move_player2[0]) - 65][int(move_player2[1]) - 1] = 'M'
        game.play()

        winner_name = game.players[0].name if game.players[0].opponent_board.are_all_ships_sunk() else game.players[1].name
        results[move_player1] += 1 if winner_name == computer_player1.name else 0
        results[move_player2] += 1 if winner_name == computer_player2.name else 0

    return results

In [ ]:
if __name__ == "__main__":
    # Player vs. Player
    player1 = HumanPlayer("Player 1")
    player2 = HumanPlayer("Player 2")
    game1 = BattleshipGame(board_size=5)
    game1.add_player(player1)
    game1.add_player(player2)
    game1.play()

In [ ]:
if __name__ == "__main__":
    # Computer vs. Player
    player3 = HumanPlayer("Player 3")
    computer_player = ComputerPlayer("Computer")
    game2 = BattleshipGame(board_size=5)
    game2.add_player(player3)
    game2.add_player(computer_player)
    game2.play()

In [ ]:
if __name__ == "__main__":
     # Computer vs. Computer
    num_games = 50
    stats = run_computer_vs_computer_stats(board_size=5, num_games=num_games)

    # Plot the statistics
    moves = list(stats.keys())
    wins = list(stats.values())

    plt.bar(moves, wins)
    plt.xlabel('First Move')
    plt.ylabel('Number of Wins')
    plt.title('Efficiency of First Moves in Computer vs. Computer Battleship')
    plt.show()